In [1]:
train_data_head_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/train_final_master_HEAD.FITS'
train_data_phot_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/train_final_master_PHOT.FITS'
test_data_head_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/test_final_master_HEAD.FITS'
test_data_phot_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/test_final_master_PHOT.FITS'

In [2]:
from astropy.table import Table
df_header = Table.read(train_data_head_path, format='fits').to_pandas()
df_phot = Table.read(train_data_phot_path, format='fits').to_pandas()

In [3]:
df_header

,SNID,SNTYPE,SUBSURVEY,IAUC,FAKE,RA,PIXSIZE,NXPIX,NYPIX,CCDNUM,...,SIM_MWEBV,SIM_PEAKMJD,SIM_MAGSMEAR_COH,SIM_AV,SIM_RV,SIM_PEAKMAG_g,SIM_PEAKMAG_r,SIM_EXPOSURE_g,SIM_EXPOSURE_r,SIM_SUBSAMPLE_INDEX
0,5081715,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
1,6596653,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
2,9086177,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
3,3393105,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
4,5800153,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22275,769000,190,b' ',b'NULL',2,1.642074e+00,1.000000e+00,-9,-9,0,...,3.627962e-02,5.759820e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,2.026069e+01,1.916512e+01,1.000000e+00,1.000000e+00,-9
22276,73480109,190,b' ',b'NULL',2,9.516176e-01,1.000000e+00,-9,-9,0,...,2.369709e-02,5.696953e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,2.076467e+01,1.968107e+01,1.000000e+00,1.000000e+00,-9
22277,132283133,190,b' ',b'NULL',2,3.806571e+00,1.000000e+00,-9,-9,0,...,2.141795e-02,5.748764e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,1.975302e+01,1.867547e+01,1.000000e+00,1.000000e+00,-9
22278,33259310,190,b' ',b'NULL',2,4.558530e+00,1.000000e+00,-9,-9,0,...,2.260945e-02,5.745557e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,1.943544e+01,1.869953e+01,1.000000e+00,1.000000e+00,-9


In [4]:

import numpy as np
np.amin(df_phot['SNID'].values)

1757

In [5]:
from kndetect.predict_features import extract_features_all_lightcurves

In [6]:
from kndetect.utils import load_pcs

In [7]:
pcs = load_pcs()

In [8]:
from kndetect.utils import get_event_type
event_types=get_event_type(list(df_header["SNID"].values), df_header, "SNID", "SNTYPE")

In [9]:
df_header

,SNID,SNTYPE,SUBSURVEY,IAUC,FAKE,RA,PIXSIZE,NXPIX,NYPIX,CCDNUM,...,SIM_MWEBV,SIM_PEAKMJD,SIM_MAGSMEAR_COH,SIM_AV,SIM_RV,SIM_PEAKMAG_g,SIM_PEAKMAG_r,SIM_EXPOSURE_g,SIM_EXPOSURE_r,SIM_SUBSAMPLE_INDEX
0,5081715,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
1,6596653,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
2,9086177,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
3,3393105,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
4,5800153,149,b'N/A',b'N/A',16959,1.000000e+20,1.000000e+20,16959,16959,16959,...,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20,16959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22275,769000,190,b' ',b'NULL',2,1.642074e+00,1.000000e+00,-9,-9,0,...,3.627962e-02,5.759820e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,2.026069e+01,1.916512e+01,1.000000e+00,1.000000e+00,-9
22276,73480109,190,b' ',b'NULL',2,9.516176e-01,1.000000e+00,-9,-9,0,...,2.369709e-02,5.696953e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,2.076467e+01,1.968107e+01,1.000000e+00,1.000000e+00,-9
22277,132283133,190,b' ',b'NULL',2,3.806571e+00,1.000000e+00,-9,-9,0,...,2.141795e-02,5.748764e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,1.975302e+01,1.867547e+01,1.000000e+00,1.000000e+00,-9
22278,33259310,190,b' ',b'NULL',2,4.558530e+00,1.000000e+00,-9,-9,0,...,2.260945e-02,5.745557e+04,0.000000e+00,-9.000000e+00,-9.000000e+00,1.943544e+01,1.869953e+01,1.000000e+00,1.000000e+00,-9


In [10]:
df_phot=df_phot[:30000]

In [ ]:
train_features_df = extract_features_all_lightcurves(df_phot, "SNID", pcs, [b'g', b'r'])

 48%|████▊     | 407/856 [00:34<00:43, 10.43it/s]

In [ ]:
from kndetect.training import append_y_true_col
train_features_df = append_y_true_col(features_df=train_features_df,
                                      prediction_type_nos=[149, 150, 151],
                                      meta_df = df_header, 
                                      meta_key_col_name = "SNID",
                                      meta_type_col_name = "SNTYPE")

In [ ]:
train_features_df

In [ ]:
from kndetect.training import train_classifier

In [ ]:
clf, features_df = train_classifier(train_features_df)